In [2]:
#Submission for Ryan Keppel (ryan.keppel2@gmail.com) for Xeek challenge--"Go with the Flow".
#Requirements. Latest version of each.
import numpy
import random
import ipyvolume

In [18]:
#Configure and run simulation. Load environment file in ENV. Output is always in sub.csv. Output consists of
# one line--"x,y,z"--at the top. The final locations of all 200,000 agents are then listed--one per line. An
# additional file, track.npy, is a Numpy file with all the tracked agents. Any agent untracked is set to
# "-1,-1,-1".
# Takes a few hours. Prints out each turn it is processing to standard out.

#Load environment file
ENV = numpy.load("./vol6_environment.npy")

#Load seed
SEED=7
random.seed(SEED)

#Generate random integer from in [0,upto)
def randi(upto):
  return random.randint (0, upto - 1)

# Configuration component (10% of grade)
XSIZE = 300
YSIZE = 300
ZSIZE = 1250
shapes = ENV.shape
if XSIZE > shapes[0] or YSIZE > shapes[1] or ZSIZE > shapes[2]:
    raise Exception("Environment file not big enough for specified size")
TURNS= 2000
AGENTSPERTURN = 100
RANDOMGEN = 0 # Agents generated randomly in Z axis out of 100
AGENTS_TRACKED = 10 # Track this number of agents that are born every turn,
 #up to all agents born each turn.

    
if AGENTS_TRACKED > AGENTSPERTURN:
    raise Exception("Agents tracked should be at most agents generated per turn")

#Move preference matrix. Dimensions (-dz, dy, dx) from agent. Since the z direction is reversed, so is dz.
# The point at dz = 0, dx and dy = 0, is ignored.
# If there is a tie in the calculated preference, one is chosen randomly.
PREFS = [

    [
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 4],
    ],
    [
        [2, 2, 2],
        [2, 3, 2],
        [2, 2, 3],
    ]
]
    
# Stochastic move matrix. Dimensions (-dz, dy, dx) from agent. Zero means no likelihood. Numbers higher represent the number of tickets the possibility has. One ticket wins. If the winning move is blocked, the process is restarted.
PROBS = [

    [
        [1, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
    ],
    [
        [2, 2, 2],
        [2, 3, 2],
        [2, 2, 2],
    ]
]

#This was not in the list, but I assume it is relevant.
RANDOMAGENTMOVE = 0 # Agents will move randomly this many times out of 100


    

agents = []
tracking = numpy.full((AGENTSPERTURN * TURNS, TURNS, 3), -1, dtype=numpy.int16)



# Initialize the cumulative probability array.
CPROBS = []
def initCprobs():
  cur = 0
  for a, x in enumerate(PROBS):
      for b, y in enumerate(x):
          for c, z in enumerate(y):
              if z == 0:
                continue
              cur += z
              CPROBS.append((cur, c - 1, b - 1, -a))
initCprobs()

# Class that represents points in space taken up or not.
class Volume:
  def startUp(self):
    self.count = 0
    self.ar = [[[False for a in range(ZSIZE)] for b in range(YSIZE)]for c in range(XSIZE)]
  def pres(self, x, y, z):
    return self.ar[x][y][z]
  def put(self, x,y,z):
    self.ar[x][y][z] = True
    self.count += 1
  def remove(self, x,y,z):
    self.ar[x][y][z] = False
    self.count -= 1
  def full(self):
    return self.count == XSIZE * YSIZE * ZSIZE

volume = Volume()
volume.startUp()

def legal(x, y, z):
    if x in range(XSIZE) and y in range(YSIZE) and z in range(ZSIZE):
        if not volume.pres(x,y,z):
            return True
    return False

# Agent class. Stores the current position and methods.
class Agent:
    def startUp(self):
      self.x = -1
      self.y = -1
      self.z = -1
      self.dead = False
    def hasRoom(self):
      for dz in range(-1, 1):
        for dy in range(-1, 2):
          for dx in range(-1, 2):
            if dz == 0 and dy == 0 and dx == 0:
              continue
            if legal(self.x + dx, self.y + dy, self.z + dz):
              return True
      return False

    def executeMove(self, dx, dy, dz):
        volume.remove(self.x, self.y, self.z)
        self.x += dx
        self.y += dy
        self.z += dz

        if not legal(self.x, self.y, self.z):
          raise Exception
        volume.put(self.x, self.y, self.z)
        if self.z == 0:
          self.dead = True
    def iter(self):
        if self.dead:
            return
        if randi(100) < RANDOMAGENTMOVE:
             if not self.hasRoom():
                self.dead = True
                return

             self.randMove()
             return
        self.prefMove()
        
    def prefMove(self):
        maxn = -1
        maxes = []
        for ai, aa in enumerate(PREFS):
            for bi, bb in enumerate(aa):
                for ci, pre in enumerate(bb):
                    dx = ci - 1
                    dy = bi - 1
                    dz = -ai
                    if dx == 0 and dy == 0 and dz == 0:
                      continue
                    tx = self.x + dx
                    ty = self.y + dy
                    tz = self.z + dz
                    if not legal(tx, ty, tz):
                        continue
                    v = pre
                    v += ENV[tx][ty][tz]
                    if v >= maxn:
                        maxes.append((dx, dy, dz))
                        maxn = v
        if len(maxes) == 0:
          self.dead = True
          return
        cr = randi(len(maxes))
        choose = maxes[cr]
        (dx, dy, dz) = choose
        self.executeMove(dx, dy, dz)

    def randMove(self):

        while True:
            choice = randi(CPROBS[-1][0])
            for cp in CPROBS:
              if choice < cp[0]:
                g, dx, dy, dz = cp
                if legal(self.x + dx, self.y + dy, self.z + dz):
                  self.executeMove(dx, dy, dz)
                  return
                break
        

agents = []

# Run the simulation
def process():
    for turn in range(TURNS):
        print("turn ", turn)
        for agent in range(AGENTSPERTURN):
            agent = Agent()
            agent.startUp()
            
            while True:
                if volume.full():
                  raise Exception
                x = randi(XSIZE)
                y = randi(YSIZE)
                if randi(100) < RANDOMGEN:
                  z = randi(ZSIZE)
                else:
                  z = ZSIZE - 1
                if legal(x, y, z):
                   break
            agent.x, agent.y, agent.z = x, y, z
            if agent.z == 0:
              agent.dead = True
            volume.put(agent.x, agent.y, agent.z)
            agents.append(agent)
        for k, agent in enumerate(agents):
           if k % AGENTSPERTURN < AGENTS_TRACKED:
             tracking[k, turn] = [agent.x, agent.y, agent.z]
           agent.iter()

#This file contains the last x,y,z of all agents (dead or alive)
f = open('sub.csv', 'w')
print("x,y,z", file=f)
process()
for agent in agents:
  print(agent.x, agent.y, agent.z, sep=',', file=f)

f.close()
#Tracking file is always saved, but only updated according to the agents tracked.
numpy.save('track.npy', tracking)


turn  0
turn  1
turn  2
turn  3
turn  4
turn  5
turn  6
turn  7
turn  8
turn  9
turn  10
turn  11
turn  12
turn  13
turn  14
turn  15
turn  16
turn  17
turn  18
turn  19
turn  20
turn  21
turn  22
turn  23
turn  24
turn  25
turn  26
turn  27
turn  28
turn  29
turn  30
turn  31
turn  32
turn  33
turn  34
turn  35
turn  36
turn  37
turn  38
turn  39
turn  40
turn  41
turn  42
turn  43
turn  44
turn  45
turn  46
turn  47
turn  48
turn  49
turn  50
turn  51
turn  52
turn  53
turn  54
turn  55
turn  56
turn  57
turn  58
turn  59
turn  60
turn  61
turn  62
turn  63
turn  64
turn  65
turn  66
turn  67
turn  68
turn  69
turn  70
turn  71
turn  72
turn  73
turn  74
turn  75
turn  76
turn  77
turn  78
turn  79
turn  80
turn  81
turn  82
turn  83
turn  84
turn  85
turn  86
turn  87
turn  88
turn  89
turn  90
turn  91
turn  92
turn  93
turn  94
turn  95
turn  96
turn  97
turn  98
turn  99
turn  100
turn  101
turn  102
turn  103
turn  104
turn  105
turn  106
turn  107
turn  108
turn  109
turn  110


turn  831
turn  832
turn  833
turn  834
turn  835
turn  836
turn  837
turn  838
turn  839
turn  840
turn  841
turn  842
turn  843
turn  844
turn  845
turn  846
turn  847
turn  848
turn  849
turn  850
turn  851
turn  852
turn  853
turn  854
turn  855
turn  856
turn  857
turn  858
turn  859
turn  860
turn  861
turn  862
turn  863
turn  864
turn  865
turn  866
turn  867
turn  868
turn  869
turn  870
turn  871
turn  872
turn  873
turn  874
turn  875
turn  876
turn  877
turn  878
turn  879
turn  880
turn  881
turn  882
turn  883
turn  884
turn  885
turn  886
turn  887
turn  888
turn  889
turn  890
turn  891
turn  892
turn  893
turn  894
turn  895
turn  896
turn  897
turn  898
turn  899
turn  900
turn  901
turn  902
turn  903
turn  904
turn  905
turn  906
turn  907
turn  908
turn  909
turn  910
turn  911
turn  912
turn  913
turn  914
turn  915
turn  916
turn  917
turn  918
turn  919
turn  920
turn  921
turn  922
turn  923
turn  924
turn  925
turn  926
turn  927
turn  928
turn  929
turn  930


turn  1592
turn  1593
turn  1594
turn  1595
turn  1596
turn  1597
turn  1598
turn  1599
turn  1600
turn  1601
turn  1602
turn  1603
turn  1604
turn  1605
turn  1606
turn  1607
turn  1608
turn  1609
turn  1610
turn  1611
turn  1612
turn  1613
turn  1614
turn  1615
turn  1616
turn  1617
turn  1618
turn  1619
turn  1620
turn  1621
turn  1622
turn  1623
turn  1624
turn  1625
turn  1626
turn  1627
turn  1628
turn  1629
turn  1630
turn  1631
turn  1632
turn  1633
turn  1634
turn  1635
turn  1636
turn  1637
turn  1638
turn  1639
turn  1640
turn  1641
turn  1642
turn  1643
turn  1644
turn  1645
turn  1646
turn  1647
turn  1648
turn  1649
turn  1650
turn  1651
turn  1652
turn  1653
turn  1654
turn  1655
turn  1656
turn  1657
turn  1658
turn  1659
turn  1660
turn  1661
turn  1662
turn  1663
turn  1664
turn  1665
turn  1666
turn  1667
turn  1668
turn  1669
turn  1670
turn  1671
turn  1672
turn  1673
turn  1674
turn  1675
turn  1676
turn  1677
turn  1678
turn  1679
turn  1680
turn  1681
turn  1682

In [8]:
#Visualization of environment (3D, animated)
env = numpy.load("./vol6_environment.npy")
ipyvolume.pylab.volshow(env)
ipyvolume.pylab.show()

In [20]:
#Visualization of agents at end of simulation. Run the simulation first to generate
# the .csv file.
file = open("sub.csv")
locs = numpy.loadtxt(file,delimiter = ',',skiprows=1,
                    dtype=numpy.uint16).T
ipyvolume.quickscatter(locs[0], locs[1], locs[2],size=1)
ipyvolume.xlim(0, 299)
ipyvolume.ylim(0, 299)
ipyvolume.zlim(0, 1249)
ipyvolume.show()

In [19]:
#Visualization of the track of a given agent. The agent ID is turn_born * AGENTSPERTURN
# + n'th(zero based) agent born that turn. n must be less than the agents tracked per turn (AGENTS_TRACKED)
# to have a valid path.
track = numpy.load("track.npy")
turn = 0
n = 3
id = turn*100 + n
ipyvolume.quickscatter(track[id,:,0], track[id,:,1], track[id,:,2])
ipyvolume.xlim(0, 299)
ipyvolume.ylim(0, 299)
ipyvolume.zlim(0, 1249)
ipyvolume.show()

In [7]:
# This was used to visualize the accumulations file. Not part of the scoring criteria.
n = numpy.load("vol5_accumulations.npy")
n2 = numpy.where(n == 1)

ipyvolume.quickscatter(n2[0].astype(numpy.uint16),n2[1].astype(numpy.uint16),n2[2].astype(numpy.uint16))
ipyvolume.xlim(0, 299)
ipyvolume.ylim(0, 299)
ipyvolume.zlim(0, 1249)
ipyvolume.show()

In [11]:
import ipywidgets
ipywidgets.IntSlider()

IntSlider(value=0)

In [10]:
ipywidgets.IntSlider()

IntSlider(value=0)